In [1]:
#!pip install -r requirements.txt

In [2]:
import pandas as pd
import torch

from os import path
from pathlib import Path
from transformers import DistilBertModel, DistilBertTokenizerFast, DistilBertTokenizer
from transformers import Trainer, TrainingArguments
from datasets import load_dataset, load_metric

torch.cuda.empty_cache()

In [3]:
shake_data_path = "data/shakespeare/"
debugging = pd.read_csv(shake_data_path+'train.csv').sample(5, random_state=5)

# initalize paths
# if Path("path").exists():
#     print('path exists')
# else:
#     print('creating path')
#     path_obj = Path("path")
#     path_obj.mkdir()

In [4]:
# for more info on splitting datasets https://huggingface.co/docs/datasets/v1.4.1/splits.html
data_files = {'train': shake_data_path+'train.csv', 'eval': shake_data_path+'dev.csv'}
test_dataset = load_dataset('csv', data_files={'test': shake_data_path+'test.csv'})
dataset = load_dataset("csv", data_files=data_files)

Using custom data configuration default-4d1ab6618f592634


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to C:\Users\User\.cache\huggingface\datasets\csv\default-4d1ab6618f592634\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-408e6f335692ad10


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to C:\Users\User\.cache\huggingface\datasets\csv\default-408e6f335692ad10\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['play_line_id', 'title', 'line', 'original_text', 'modern_text', 'original_text_cleaned', 'modern_text_cleaned', 'combined_text_clean'],
        num_rows: 16863
    })
    eval: Dataset({
        features: ['play_line_id', 'title', 'line', 'original_text', 'modern_text', 'original_text_cleaned', 'modern_text_cleaned', 'combined_text_clean'],
        num_rows: 2108
    })
})

In [5]:
model_name = "distilbert-base-uncased"
fast_tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertModel.from_pretrained(model_name)

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
# train a new tokenizer
def get_training_corpus():
    return (
        dataset["train"][i:i+1000]["original_text_cleaned"]
        for i in range(0, len(dataset["train"]), 1000)
    )
training_corpus = get_training_corpus()

og_str = """"""
for line in debugging.original_text_cleaned.values:
    og_str = og_str+line+' '
    
md_str = """"""
for line in debugging.modern_text_cleaned.values:
    md_str = md_str+line+' '

trained_tokenizer = fast_tokenizer.train_new_from_iterator(training_corpus, 52000)
og_tokens_bert = fast_tokenizer.tokenize(og_str)
og_tokens_trained = trained_tokenizer.tokenize(og_str)
print(og_str+"\n-----------------------------------------------------")
print(og_tokens_bert)
print("-----------------------------------------------------")
print(og_tokens_trained)

# a new tokenizer doesn't really seem necessary based on this output

NameError: name 'fast_tokenizer' is not defined

In [9]:
tokenized_train_dataset = dataset['train'].map(lambda x: tokenizer(x['combined_text_clean'],padding = 'max_length', max_length =512, truncation=True))
tokenized_dev_dataset = dataset['eval'].map(lambda x: tokenizer(x['combined_text_clean'],padding = 'max_length', max_length =512, truncation=True))
tokenized_test_dataset = test_dataset['test'].map(lambda x: tokenizer(x['combined_text_clean'],padding = 'max_length', max_length =512, truncation=True))

  0%|          | 0/16863 [00:00<?, ?ex/s]

  0%|          | 0/2108 [00:00<?, ?ex/s]

  0%|          | 0/2108 [00:00<?, ?ex/s]

In [12]:
# EncoderDecoder Example
from transformers import EncoderDecoderModel, BertTokenizer

model = EncoderDecoderModel.from_encoder_decoder_pretrained(
    "bert-base-uncased", "bert-base-uncased"
)  # initialize Bert2Bert from pre-trained checkpoints

# training
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.decoder.vocab_size

input_ids = tokenizer(og_str, return_tensors="pt").input_ids
labels = tokenizer(md_str, return_tensors="pt").input_ids
outputs = model(input_ids=input_ids, labels=input_ids)
loss, logits = outputs.loss, outputs.logits

# save and load from pretrained
model.save_pretrained("test_bert2bert")
model = EncoderDecoderModel.from_pretrained("test_bert2bert")

# generation
generated = model.generate(input_ids) # failed, need to try on greatlakes

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relations

c:\users\user\documents\projects\si630_finalproject\venv_final_proj\lib\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:531: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\caffe2\serialize\inline_container.cc:300] . unexpected pos 145872768 vs 145872656

In [8]:
metric = load_metric("bleu")
training_args = TrainingArguments(
    output_dir = 'testing'
)

In [8]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_dev_dataset,
    compute_metrics= metric
)

In [9]:
trainer.train()

ValueError: Trainer: training requires a train_dataset.